In [1]:
# Dependencies
import numpy as np
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [2]:
import sqlite3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
engine = create_engine('sqlite:///db/newfinaldata.sqlite')
# Declare a Base using `automap_base()`
Base = automap_base()
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [4]:
# Save references to each table
Movies = Base.classes.movie_data
Images = Base.classes.images
Upcoming = Base.classes.upcoming

In [5]:
# Create a session
session = Session(engine)

In [6]:
cnx = sqlite3.connect('db/newfinaldata.sqlite')
df_upcoming = pd.read_sql_query("SELECT * FROM upcoming", cnx)
df_upcoming["name"][6]

'The Kitchen'

In [7]:
# df_upcoming['genre'] = df_upcoming['genre'].str.strip()
# df_upcoming['genre'] = df_upcoming['genre'].str.replace(" ","")
# df_upcoming["genre"][10]

In [8]:
def recommend_upcoming(movie_name, genre):

    cnx = sqlite3.connect('db/newfinaldata.sqlite')
    df_upcoming = pd.read_sql_query("SELECT * FROM upcoming", cnx)
    
    # drop unnecessary column
    df_upcoming = df_upcoming[['name', 'genre']]
    df_upcoming['genre'] = df_upcoming['genre'].str.strip()
    df_upcoming['genre'] = df_upcoming['genre'].str.replace(" ","")
    
    
#     print(df_upcoming.head())
#     print(df_upcoming["genre"])

    # Break up the big genre string into a string array
    df_upcoming['genre'] = df_upcoming['genre'].str.split(',')
    
#     print(df_upcoming.head())

    dict1 = {
         "name": movie_name['Title'],
         "genre": genre
    }
    
#     print(dict1)
    
    ref_df = pd.DataFrame(dict1, index = [0])
    ref_df['genre'] = ref_df['genre'].str.split('|')
#     ref_df['genre'] = ref_df['genre'].fillna("").astype('str')
#     print(ref_df)
    
    
    df_upcoming = df_upcoming.append(ref_df, ignore_index=True)
    
    print(df_upcoming.tail())

    # Convert genres to string value
    df_upcoming['genre'] = df_upcoming['genre'].fillna("").astype('str')
    print(df_upcoming.tail())

    tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    tfidf_matrix = tf.fit_transform(df_upcoming['genre'])
    # print(tfidf_matrix)
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    # print(cosine_sim)
    

    # Build a 1-dimensional array with movie titles
    titles = df_upcoming['name']
    indices = pd.Series(df_upcoming.index, index=df_upcoming['name'])
    # print(indices)

    idx = indices[movie_name['Title']]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:4]
    movie_indices = [i[0] for i in sim_scores]

    recommendations =  titles.iloc[movie_indices]

    return recommendations

In [9]:
def get_genre(movie):
    sel = [
        Movies.name,
        Movies.genre
    ]

    table = session.query(*sel).filter(Movies.name == movie).all()
#     print(table)

    movie_data = []
    for results in table:
        movie = {}
        movie["Title"] = results[0]
        movie["Genre"] = results[1]
        movie_data.append(movie)
    
#     print(movie_data)
    movie_data = movie_data[0]
    
    # movie_data['Genre'] = movie_data['Genre'].split("|")
#     print(movie_data['Genre'])

    upcoming_movies = list(recommend_upcoming(movie, movie_data["Genre"]))
#     print(type(upcoming_movies))
    
    counter = 0
    for item in upcoming_movies:
        if item == movie_data["Title"]:
            upcoming_movies.remove(movie_data["Title"])
            counter += 1

    if counter == 0:
        del upcoming_movies[-1]
        
    return upcoming_movies
#     movie_recommendation = clean_movies_two(upcoming_movies, movie)

#     try:     
#         movie_recommendation = clean_movies_two(upcoming_movies, movie)
#     except KeyError:
#         movie_recommendation: []

#     return movie_recommendation


In [10]:
get_genre('10 Cloverfield Lane')

                   name                                       genre
50    The Rosie Project                                   [Romance]
51                Wendy                            [Drama, Fantasy]
52                Spawn                   [Action, Fantasy, Horror]
53                  Eli                                    [Horror]
54  10 Cloverfield Lane  [Drama, Horror, Mystery, Sci-Fi, Thriller]
                   name                                              genre
50    The Rosie Project                                        ['Romance']
51                Wendy                               ['Drama', 'Fantasy']
52                Spawn                    ['Action', 'Fantasy', 'Horror']
53                  Eli                                         ['Horror']
54  10 Cloverfield Lane  ['Drama', 'Horror', 'Mystery', 'Sci-Fi', 'Thri...


['Ready or Not', 'Black Christmas', 'Lucy in the Sky']